In [23]:
import pandas as pd
import numpy as np
import scipy.stats as stats
from numpy.linalg import inv

In [24]:
def clean_data(df):
    for col in df.columns:
        if df[col].dtype == 'object':
            df[col] = df[col].str.replace(",", "").str.replace("$", "").astype(float)
    return df.dropna()

In [25]:
# Read data
data = pd.read_csv("C:/Users/Luxford/Documents/Lesson/AD/factbookv2.csv")

# Filter and process the data
data_dipake = ["GDP", "Exports", "Imports", "Industrial production growth rate", "Investment", "Unemployment rate"]
df = clean_data(data[data_dipake])

# Check for missing values and display the first five rows
print(df.isnull().sum())
df.head()

GDP                                  0
Exports                              0
Imports                              0
Industrial production growth rate    0
Investment                           0
Unemployment rate                    0
dtype: int64


C:\Users\Luxford\AppData\Local\Temp\ipykernel_16256\4068154764.py:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df[col] = df[col].str.replace(",", "").str.replace("$", "").astype(float)
C:\Users\Luxford\AppData\Local\Temp\ipykernel_16256\4068154764.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].str.replace(",", "").str.replace("$", "").astype(float)


,GDP,Exports,Imports,Industrial production growth rate,Investment,Unemployment rate
0,1.746000e+10,5.524000e+08,2.076000e+09,3.1,18.4,14.8
1,2.123000e+11,3.216000e+10,1.525000e+10,6.0,26.2,25.4
3,4.835000e+11,3.378000e+10,2.206000e+10,12.0,18.3,14.8
4,1.365000e+10,8.500000e+08,1.300000e+09,15.0,19.8,30.0
5,6.117000e+11,8.689000e+10,9.810000e+10,1.9,25.3,5.1


In [26]:
def create_X(df, *vars):
    n = df['GDP'].count()
    X = np.array([np.ones(n)] + [df[var] for var in vars]).T
    return X

def calculate_b(X, Y):
    return np.matmul(np.matmul(inv(np.matmul(X.T, X)), X.T), Y)

def calculate_y_hat(X, b):
    return np.matmul(X, b)

def calculate_Rb(y_hat, y_avg):
    return np.matmul((y_hat - y_avg).T, (y_hat - y_avg))

def calculate_f(Rb, sse, n, k):
    return Rb / (sse / (n - k - 1))


In [27]:
def R_vars(df, *vars):
    n = df['GDP'].count()
    Y = np.array([df['GDP']]).T
    y_avg = np.average(Y)
    X = create_X(df, *vars)
    b = calculate_b(X, Y)
    y_hat = calculate_y_hat(X, b)
    Rb = calculate_Rb(y_hat, y_avg)
    return Rb, y_hat

In [28]:
# Calculate Rb and y_hat for each model
Rb1, y_hat1 = R_vars(df, 'Exports')
Rb2, y_hat2 = R_vars(df, 'Imports')
Rb3, y_hat3 = R_vars(df, 'Industrial production growth rate')
Rb4, y_hat4 = R_vars(df, 'Investment')
Rb5, y_hat5 = R_vars(df, 'Unemployment rate')

In [29]:
print(Rb1)
print(Rb2)
print(Rb3)
print(Rb4)
print(Rb5)

[[1.28742661e+26]]
[[1.7256398e+26]]
[[2.4349925e+23]]
[[6.95704935e+23]]
[[5.23362087e+24]]


In [30]:
Y = np.array([df['GDP']]).T
n = df['GDP'].count()

In [ ]:
# Check whether b2 is significant or not
k = 1

sse2 = np.matmul((Y - y_hat2).T, (Y - y_hat2))
print(f"MSE Model 2: {sse2}")
f2 = Rb2/(sse2 / (n - k - 1))
print(f"f dari Model 2: {f2}")
f_table = stats.f.ppf(0.95, 1, n-k-1)
print(f"f dari tabel: {f_table}")


MSE Model 2: [[4.49802423e+25]]
f dari Model 2: [[437.35410609]]
f dari tabel: 3.924330484639666


So, because f model 2 > f table, then variable 2 has a significant influence

In [32]:
# Calculate Rb and y_hat for combined models
Rb21, y_hat21 = R_vars(df, 'Imports', 'Exports')
Rb23, y_hat23 = R_vars(df, 'Imports', 'Industrial production growth rate')
Rb24, y_hat24 = R_vars(df, 'Imports', 'Investment')
Rb25, y_hat25 = R_vars(df, 'Imports', 'Unemployment rate')

In [33]:
print(Rb21)
print(Rb23)
print(Rb24)
print(Rb25)

[[1.76619881e+26]]
[[1.74719666e+26]]
[[1.73664248e+26]]
[[1.72864702e+26]]


In [ ]:
# Check whether the addition of b1 is significant or not (R(b1 | b2))
# (R(b1 | b2) = R(b1, b2) - R(b2))
k = 2

sse21 = np.matmul((Y - y_hat21).T, (Y - y_hat21))
print(f"MSE Model B1: {sse21}")
f21 = (Rb21 - Rb2)/(sse21 / (n - k - 1))
print(f"f penambahan variabel 1 pada Model B1: {f21}")
f_table = stats.f.ppf(0.95, 1, n-k-1)
print(f"f dari tabel: {f_table}")


MSE Model B1: [[4.09243406e+25]]
f penambahan variabel 1 pada Model B1: [[11.19912731]]
f dari tabel: 3.9250756165304113


In [ ]:
# Check whether b2 is still significant or not, after adding b1
# (R(b2 | b1) = R(b2, b1) - R(b1))
k = 2

sse21 = np.matmul((Y - y_hat21).T, (Y - y_hat21))
print(f"MSE Model B1: {sse21}")
f21 = (Rb21 - Rb1)/(sse21 / (n - k - 1))
print(f"f b2 setelah ditambahkan b1 Model B1: {f21}")
f_table = stats.f.ppf(0.95, 1, n-k-1)
print(f"f dari tabel: {f_table}")

MSE Model B1: [[4.09243406e+25]]
f b2 setelah ditambahkan b1 Model B1: [[132.19824267]]
f dari tabel: 3.9250756165304113


Since f model B1 > f table, after adding variable 1, variable 2 is still significant.
This means we use variables 1 and 2.

In [36]:
# Calculate Rb and y_hat for 3-variable models
Rb213, y_hat213 = R_vars(df, 'Imports', 'Exports', 'Industrial production growth rate')
Rb214, y_hat214 = R_vars(df, 'Imports', 'Exports', 'Investment')
Rb215, y_hat215 = R_vars(df, 'Imports', 'Exports', 'Unemployment rate')

In [37]:
print(Rb213)
print(Rb214)
print(Rb215)

[[1.78669291e+26]]
[[1.77947588e+26]]
[[1.76714336e+26]]


In [ ]:
# Check whether the addition of b3 is significant or not (R(b3 | b2, b1))
# (R(b3 | b1, b2) = R(b1, b2, b3) - R(b1, b2))
k = 3

sse312 = np.matmul((Y - y_hat213).T, (Y - y_hat213))
print(f"MSE Model C1: {sse312}")
f312 = (Rb213 - Rb21)/(sse312 / (n - k - 1))
print(f"f penambahan variabel 3 pada Model C1: {f312}")
f_table = stats.f.ppf(0.95, 1, n-k-1)
print(f"f dari tabel: {f_table}")


MSE Model C1: [[3.88749308e+25]]
f penambahan variabel 3 pada Model C1: [[5.90441939]]
f dari tabel: 3.9258342687862307


In [ ]:
# Check whether b1 is still significant after b3 is added.
# (R(b1 | b2, b3) = R(b1, b2, b3) - R(b2, b3))
k = 3

sse123 = np.matmul((Y - y_hat213).T, (Y - y_hat213))
print(f"MSE Model C1: {sse123}")
f123 = (Rb213 - Rb23)/(sse123 / (n - k - 1))
print(f"f b1 setelah ditambahkan b2, b3 Model C1: {f123}")
f_table = stats.f.ppf(0.95, 1, n-k-1)
print(f"f dari tabel: {f_table}")

MSE Model C1: [[3.88749308e+25]]
f b1 setelah ditambahkan b2, b3 Model C1: [[11.37900419]]
f dari tabel: 3.9258342687862307


In [40]:
def R_2vars(df, var1, var2):
    n = df['GDP'].count()
    Y = np.array([df['GDP']]).T
    y_avg = np.average(Y)

    X12 = np.array([np.ones(n), df[var1], df[var2]]).T
    b12 = np.matmul(np.matmul(inv(np.matmul(X12.T, X12)), X12.T), Y)
    y_hat12 = np.matmul(X12, b12) # Prediksi Y (GDP) menggunakan model 1
    Rb12 = np.matmul((y_hat12 - y_avg).T, (y_hat12 - y_avg))
    return Rb12, y_hat12

In [ ]:
# Check whether b2 is still significant after b3 is added.
# (R(b2 | b1, b3) = R(b1, b2, b3) - R(b1, b3))
k = 3

Rb13, _ = R_2vars(df, "Exports", "Industrial production growth rate")

sse213 = np.matmul((Y - y_hat213).T, (Y - y_hat213))
print(f"MSE Model C1: {sse213}")
f213 = (Rb213 - Rb13)/(sse213 / (n - k - 1))
print(f"f b2 setelah ditambahkan b1, b3 Model C1: {f213}")
f_table = stats.f.ppf(0.95, 1, n-k-1)
print(f"f dari tabel: {f_table}")

MSE Model C1: [[3.88749308e+25]]
f b2 setelah ditambahkan b1, b3 Model C1: [[138.58263945]]
f dari tabel: 3.9258342687862307


Because b1, b2 are still significant after adding b3, the model becomes b1, b2, b3

In [42]:
# Calculate Rb and y_hat for 4-variable models
Rb2134, y_hat2134 = R_vars(df, 'Imports', 'Exports', 'Industrial production growth rate', 'Investment')
Rb2135, y_hat2135 = R_vars(df, 'Imports', 'Exports', 'Industrial production growth rate', 'Unemployment rate')

In [43]:
print(Rb2134)
print(Rb2135)

[[1.79315321e+26]]
[[1.78767105e+26]]


In [ ]:
# Check whether the addition of b4 is significant or not (R(b4 | b1, b2, b3))
# (R(b4 | b1, b2, b3) = R(b1, b2, b3, b4) - R(b1, b2, b3))
k = 4

sse41234 = np.matmul((Y - y_hat2134).T, (Y - y_hat2134))
print(f"MSE Model D1: {sse41234}")
f41234 = (Rb2134 - Rb213)/(sse41234 / (n - k - 1))
print(f"f b4 setelah penambahan 4 variabel pada Model D1: {f41234}")
f_table = stats.f.ppf(0.95, 1, n-k-1)
print(f"f dari tabel: {f_table}")

MSE Model D1: [[3.82289012e+25]]
f b4 setelah penambahan 4 variabel pada Model D1: [[1.87578719]]
f dari tabel: 3.926606812744345


In [ ]:
# Check whether the addition of b5 is significant or not (R(b5 | b1, b2, b3))
# (R(b5 | b1, b2, b3) = R(b1, b2, b3, b5) - R(b1, b2, b3))
k = 4

sse2135 = np.matmul((Y - y_hat2135).T, (Y - y_hat2135))
print(f"MSE Model D2: {sse2135}")
f2135 = (Rb2135 - Rb213)/(sse2135 / (n - k - 1))
print(f"f b5 setelah penambahan 4 variabel pada Model D2: {f2135}")
f_table = stats.f.ppf(0.95, 1, n-k-1)
print(f"f dari tabel: {f_table}")

MSE Model D2: [[3.87771164e+25]]
f b5 setelah penambahan 4 variabel pada Model D2: [[0.27999489]]
f dari tabel: 3.926606812744345


Because the f-value of adding b4 and b5 < f-table, variables b4 and b5 are not significant when added to the previously obtained model.
So the model used is b1, b2, b3

In [46]:
# Calculate final model coefficients
bfinal = calculate_b(create_X(df, 'Exports', 'Imports', 'Industrial production growth rate'), np.array([df['GDP']]).T)
print(bfinal)

[[-1.45787120e+11]
 [-3.24013692e+00]
 [ 9.51317989e+00]
 [ 2.37756770e+10]]


Final Result:
- b0 = -1.46 E11
- b1 = -3.24
- b2 =  9.51
- b3 =  2.38 E10

Linear model equation:
Y = -1.46*(10^11) - 3.24 * X1 + 9.51 * X2 + 2.38*(10^10) * X3